In [1]:
#calculo
import numpy as np
import pandas as pd

#grafico
import matplotlib.pyplot as plt
%matplotlib inline

#propios utilitarios
from utils import *

data = pd.read_csv('data/01dataBaseTrainTrxRec.csv', **set_parameter_csv)
data.head()

,fechaOper,codCliente,codGiro,codEstab,flagLimaProvEstab,ubigeoEstab,ctdTrx,ratingMonto
0,2017-01-29 00:00:00,7649,138.0,43629,1,176.0,1,0.014072
1,2016-12-01 00:00:00,24604,75.0,4326,0,81.0,1,0.001667
2,2017-06-01 00:00:00,15289,75.0,4326,0,81.0,1,0.000127
3,2017-09-22 00:00:00,5190,110.0,59776,1,156.0,1,0.001167
4,2017-05-05 00:00:00,16635,75.0,31043,0,81.0,1,0.047386


In [2]:
null_verificator(data)

,Nulos,Cantidad,Tipo Col
fechaOper,False,0,object
codCliente,False,0,int64
codGiro,True,92686,float64
codEstab,False,0,int64
flagLimaProvEstab,False,0,int64
ubigeoEstab,True,92686,float64
ctdTrx,False,0,int64
ratingMonto,False,0,float64


In [4]:
data.shape, get_memory_usage(data)

((1591617, 8), '200.36079788208008 MB')

In [5]:
reduce_size_data(data)

In [6]:
get_memory_usage(data)

'88.07358360290527 MB'

In [7]:
data.describe(include='all')

,fechaOper,codCliente,codGiro,codEstab,flagLimaProvEstab,ubigeoEstab,ctdTrx,ratingMonto
count,1591617,1.591617e+06,1.498931e+06,1.591617e+06,1.591617e+06,1.498931e+06,1.591617e+06,1.591617e+06
unique,365,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,2017-07-15 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,7303,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,1.334583e+04,1.048248e+02,2.453260e+04,8.084175e-01,1.747467e+02,1.072021e+00,1.318383e-02
std,NaN,8.493548e+03,3.920585e+01,1.990371e+04,3.935464e-01,6.856421e+01,3.465646e-01,3.236438e-02
min,NaN,1.000000e+00,2.000000e+00,1.000000e+00,0.000000e+00,2.000000e+00,1.000000e+00,2.984905e-06
25%,NaN,6.035000e+03,7.500000e+01,4.339000e+03,1.000000e+00,1.480000e+02,1.000000e+00,1.598366e-03
50%,NaN,1.258100e+04,1.080000e+02,2.266500e+04,1.000000e+00,1.650000e+02,1.000000e+00,4.199563e-03
75%,NaN,2.046200e+04,1.120000e+02,3.673100e+04,1.000000e+00,1.750000e+02,1.000000e+00,1.152572e-02


In [8]:
null_verificator(data)

,Nulos,Cantidad,Tipo Col
fechaOper,False,0,category
codCliente,False,0,int64
codGiro,True,92686,float64
codEstab,False,0,int64
flagLimaProvEstab,False,0,int64
ubigeoEstab,True,92686,float64
ctdTrx,False,0,int64
ratingMonto,False,0,float64
